In [1]:
# Step 1: Regex Syntax Check
import re

def is_valid_email_regex(email: str) -> bool:
    """Quickly check if the email looks valid using regex."""
    pattern = r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"
    return bool(re.match(pattern, email))

# Example usage:
print(is_valid_email_regex("user@example.com"))  # True
print(is_valid_email_regex("bad@email"))         # False

True
False


In [2]:
# Step 2: Validate with email-validator library
from email_validator import validate_email, EmailNotValidError

def is_valid_email_library(email: str) -> str:
    """Validate email using the email-validator library. Returns normalized email or None."""
    try:
        valid = validate_email(email, check_deliverability=False)  # We'll check MX separately
        return valid.email  # Normalized email
    except EmailNotValidError as e:
        print(f"Invalid: {e}")
        return None

# Example usage:
print(is_valid_email_library("user@example.com"))  # user@example.com
print(is_valid_email_library("bad@email"))         # None

user@example.com
Invalid: The part after the @-sign is not valid. It should have a period.
None


In [3]:
# Step 3: DNS MX Record Check
import dns.resolver

def has_mx_record(domain: str) -> bool:
    """Check if the domain has at least one MX record (accepts email)."""
    try:
        answers = dns.resolver.resolve(domain, 'MX')
        return len(answers) > 0
    except Exception as e:
        print(f"No MX record found: {e}")
        return False

def is_deliverable_email(email: str) -> bool:
    """Full check: regex, library, and MX record."""
    if not is_valid_email_regex(email):
        print("Failed regex check.")
        return False
    normalized = is_valid_email_library(email)
    if not normalized:
        print("Failed library check.")
        return False
    domain = normalized.split('@')[1]
    if not has_mx_record(domain):
        print("Failed MX check.")
        return False
    return True

# Example usage:
print(is_deliverable_email("user@gmail.com"))  # Should be True
print(is_deliverable_email("nobody@invalid-domain.tld"))  # Should be False

True
No MX record found: The DNS query name does not exist: invalid-domain.tld.
Failed MX check.
False


In [4]:
# --- Real-Time Test: Try your own email here ---
test_email = "sr6172@srmist.edu.in"  # Change this to test other emails

print(f"Testing: {test_email}")
if is_deliverable_email(test_email):
    print(f"{test_email} is a valid and deliverable email!")
else:
    print(f"{test_email} is NOT valid or deliverable.")

Testing: sr6172@srmist.edu.in
sr6172@srmist.edu.in is a valid and deliverable email!


In [7]:
# Additional real-time tests to verify the approach
test_emails = [
    "sr6172@srmist.edu.in",     # Valid college email
    "user@gm.com",           # Valid Gmail
    "invalid@fake-domain.xyz",  # Invalid domain
    "bad-format@",              # Bad format
    "test@yahoo.com",           # Valid Yahoo
    "user@outlook.com" ,
    "dadqe@sadswda.com"
]

print("=== Real-Time Email Validation Test ===")
for email in test_emails:
    print(f"\nTesting: {email}")
    result = is_deliverable_email(email)
    status = "✅ VALID" if result else "❌ INVALID"
    print(f"Result: {status}")
    print("-" * 50)

=== Real-Time Email Validation Test ===

Testing: sr6172@srmist.edu.in
Result: ✅ VALID
--------------------------------------------------

Testing: user@gm.com
Result: ✅ VALID
--------------------------------------------------

Testing: invalid@fake-domain.xyz
No MX record found: The DNS query name does not exist: fake-domain.xyz.
Failed MX check.
Result: ❌ INVALID
--------------------------------------------------

Testing: bad-format@
Failed regex check.
Result: ❌ INVALID
--------------------------------------------------

Testing: test@yahoo.com
Result: ✅ VALID
--------------------------------------------------

Testing: user@outlook.com
Result: ✅ VALID
--------------------------------------------------

Testing: dadqe@sadswda.com
No MX record found: The DNS query name does not exist: sadswda.com.
Failed MX check.
Result: ❌ INVALID
--------------------------------------------------


# Industry-Style Guide to Email Validation in Python

Email validation is critical for cybersecurity, user signups, spam prevention, and API reliability. This guide covers three robust steps for real-world, production-grade email validation:

**1. Regex Syntax Check**
- Quickly filters out obviously invalid emails (e.g., missing @, bad characters).
- Fast, but not enough for production by itself.

**2. `email-validator` Library**
- Checks syntax, domain, and internationalization.
- Handles edge cases and standards (RFC compliance).
- Returns normalized, safe email addresses.

**3. DNS MX Record Check**
- Verifies the domain actually accepts email (has MX records).
- Prevents fake or typo domains from passing.

**Why this matters:**
- Stops spam and fake signups at the source.
- Ensures only real, deliverable emails are accepted.
- Essential for cybersecurity, SaaS, and any user-facing platform.

**How to use:**
- Use all three steps together for best results.
- Integrate into signup forms, APIs, or backend workflows.

---

**Next:** See the code cells below for each step, with clear explanations and real-time tests.